In [1]:
import pandas as pd

temp_df = pd.read_csv('emails.csv')
temp_df = temp_df.truncate(after=10000)

FileNotFoundError: [Errno 2] File b'emails.csv' does not exist: b'emails.csv'

In [2]:
import string
import time
from string import digits

#Two ways to make sure not to include parts with a : but not part of body
#1 make a list with all the first bits I want to ignore
exclusion_data = ['Message-ID', 'Date', 'Subject', 'Mime-Version', 'Content-Type', 'Content-Transfer-Encoding', 'X-From', 'X-To', 'X-cc', 'X-bcc', 'X-Folder', 'X-Origin', 'X-FileName', 'Cc', 'Bcc']
#2 while in the loop, create a new list by .split(":") and adding list.[0] to words not to include, but probably long, but benefit of updating?
#Solutions above aren't perfect, probably a better one to improve on later

#My dataset is being cut off at the end of new lines.

#Takes string and array of column data to keep
def extract_data(message, compulsary_data):
    email = {}
    lines = message.split('\n')
    lines = list(filter(None, lines))
    print(lines)
    filled_data = []
    body = ''
    #Go over every line
    for line in lines:
        #If the line was already added move onto the next line.
        added = False
        #Go and check for every required data
        for data in compulsary_data:
            #If the line was already used skip checking and move onto the next line
            if added:
                break
            #If the data to check was already filled then ignore it - consider just removing data from compulsary_data
            if data in filled_data:
                continue
            #If : is in the line
            if ":" in line:
                #Is either non-body data to include
                if line.startswith((data + ":")):
                    split_line = line.split(':')
                    email[data] = split_line[1].lower()
                    filled_data.append(data)
                    added = True
                #Or is body data or data to be excluded
                else:
                    #If it is part of the data to be excluded, ignore
                    ignore_line = False
                    for exclusion in exclusion_data:
                        if line.startswith((exclusion + ":")):
                            ignore_line = True
                            break
                    #If it isn't meant to be excluded then it's in the body.
                    if not ignore_line:
                        body += line + (" ")
                        added = True
            else:
                body += line + (" ")
                added = True
    #Fix this later. Forwarded emails also have data but are ignored.
    body = body.translate(str.maketrans('', '', digits))
    body = body.translate(str.maketrans('', '', '_'))
    email['Body'] = body.lower()
    if body is not "":
        filled_data.append('Body')
    #Final check, if any of the required data is missing, return nothing.
    for key in compulsary_data:
        if key not in filled_data:
            return None
        
    return email
    
def generate_df():
    compulsary_data = ['From', 'To', 'Body']
    columns = {}
    for message in temp_df['message']:
        email = extract_data(message, compulsary_data)
        if email is None:
            continue
        else:
            for key, value in email.items():
                columns.setdefault(key, []).append(value)
    return columns
   # return df

columns = generate_df()
    
#Get my columns of name, body, to, from
#Get my tfidf from it
#DBSCAN that

#tfidf by converting into matrix

#My dataset is being cut at lines?


['Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>', 'Date: Mon, 14 May 2001 16:39:00 -0700 (PDT)', 'From: phillip.allen@enron.com', 'To: tim.belden@enron.com', 'Subject: ', 'Mime-Version: 1.0', 'Content-Type: text/plain; charset=us-ascii', 'Content-Transfer-Encoding: 7bit', 'X-From: Phillip K Allen', 'X-To: Tim Belden <Tim Belden/Enron@EnronXGate>', 'X-cc: ', 'X-bcc: ', "X-Folder: \\Phillip_Allen_Jan2002_1\\Allen, Phillip K.\\'Sent Mail", 'X-Origin: Allen-P', 'X-FileName: pallen (Non-Privileged).pst', 'Here is our forecast', ' ']
['Message-ID: <15464986.1075855378456.JavaMail.evans@thyme>', 'Date: Fri, 4 May 2001 13:51:00 -0700 (PDT)', 'From: phillip.allen@enron.com', 'To: john.lavorato@enron.com', 'Subject: Re:', 'Mime-Version: 1.0', 'Content-Type: text/plain; charset=us-ascii', 'Content-Transfer-Encoding: 7bit', 'X-From: Phillip K Allen', 'X-To: John J Lavorato <John J Lavorato/ENRON@enronXgate@ENRON>', 'X-cc: ', 'X-bcc: ', "X-Folder: \\Phillip_Allen_Jan2002_1\\Allen, Phill

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [3]:
df = pd.DataFrame(columns)

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import pos_tag

#Documentation says not to do 'english' for stop_words?
stop_words = set(stopwords.words('english'))

vectorizer = TfidfVectorizer(stop_words=stop_words)
tfidf = vectorizer.fit_transform(df['Body'])
dense_tfidf = tfidf.todense()
#Remember to remove vectorizer names
print(vectorizer.get_feature_names())

import nltk
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer

def cleanup(lis):
    wnl = WordNetLemmatizer()
    lis = [wnl.lemmatize(x) for x in lis]
    return lis
    
text = cleanup(text)
print(text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\djbm12\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['aa', 'aaa', 'aachen', 'aadvantage', 'aagustin', 'aaii', 'aaiitu', 'aamir', 'aand', 'aanoli', 'aantie', 'aapl', 'aaquila', 'aaron', 'aaronow', 'aaronson', 'aarp', 'aatk', 'ab', 'aba', 'abaginski', 'abandon', 'abandoned', 'abandoning', 'abandonment', 'abaout', 'abargeri', 'abated', 'abatement', 'abb', 'abbott', 'abbreviated', 'abbreviation', 'abc', 'abel', 'aber', 'abgenix', 'abgx', 'abide', 'abiding', 'abilities', 'ability', 'abilty', 'abit', 'abl', 'able', 'abler', 'ableson', 'abn', 'abnormal', 'abo', 'aboard', 'abolish', 'abolishing', 'abort', 'abortion', 'abou', 'aboudaher', 'abound', 'aboutframe', 'abouts', 'aboutsponsorship', 'aboutwinebid', 'abovenet', 'aboves', 'abraham', 'abramo', 'abrams', 'abreast', 'abreu', 'abridged', 'abroad', 'abrogate', 'abrupt', 'abs', 'absence', 'absences', 'absent', 'absentee', 'abshire', 'ab

NameError: name 'text' is not defined

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
pca = PCA(n_components=2).fit(dense_tfidf)
X = pca.transform(dense_tfidf)
plt.scatter(X[:,0], X[:,1])
plt.show()

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn import metrics
db = DBSCAN(eps=0.3, min_samples=10).fit(tfidf)

In [ ]:
print(db.labels_)

In [ ]:
import numpy as np

core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_
    
# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

# #############################################################################
# Plot result
import matplotlib.pyplot as plt

# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = (labels == k)
    xy = X[class_member_mask & core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=14)

    xy = X[class_member_mask & ~core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=6)

plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()

In [ ]:
#More things to do:
#lemmatize
#stem
#stopwords
#noisy labels are assigned -1, so make sure to differentiate them from the other colors
# Now we print the top terms across all documents.

#Y are labels

def top_feats_per_cluster(X, y, features, min_tfidf=0.1, top_n=25):
    dfs = []

    labels = np.unique(y)
    for label in labels:
        ids = np.where(y==label) 
        feats_df = top_mean_feats(X, features, ids, min_tfidf=min_tfidf, top_n=top_n)
        feats_df.label = label
        dfs.append(feats_df)
return dfs

X = vect.fit_transform(email_df.body)
features = vect.get_feature_names()

print(top_mean_feats(X, features, None, 0.1, 10))

def top_mean_feats(X, features, grp_ids=None, min_tfidf=0.1, top_n=25):
    if grp_ids:
        D = X[grp_ids].toarray()
    else:
        D = X.toarray()

    D[D < min_tfidf] = 0
    tfidf_means = np.mean(D, axis=0)
    
return top_tfidf_feats(tfidf_means, features, top_n)


In [ ]:
#I need to convert my tfidf into a 2d array of x and ys

In [ ]:

    dbscan = cluster.DBSCAN(eps=params['eps'])
       for name, algorithm in clustering_algorithms:
        t0 = time.time()

        # catch warnings related to kneighbors_graph
        with warnings.catch_warnings():
            warnings.filterwarnings(
                "ignore",
                message="the number of connected components of the " +
                "connectivity matrix is [0-9]{1,2}" +
                " > 1. Completing it to avoid stopping the tree early.",
                category=UserWarning)
            warnings.filterwarnings(
                "ignore",
                message="Graph is not fully connected, spectral embedding" +
                " may not work as expected.",
                category=UserWarning)
            algorithm.fit(X)

        t1 = time.time()
        if hasattr(algorithm, 'labels_'):
            y_pred = algorithm.labels_.astype(np.int)
        else:
            y_pred = algorithm.predict(X)

        plt.subplot(len(datasets), len(clustering_algorithms), plot_num)
        if i_dataset == 0:
            plt.title(name, size=18)

        colors = np.array(list(islice(cycle(['#377eb8', '#ff7f00', '#4daf4a',
                                             '#f781bf', '#a65628', '#984ea3',
                                             '#999999', '#e41a1c', '#dede00']),
                                      int(max(y_pred) + 1))))
        # add black color for outliers (if any)
        colors = np.append(colors, ["#000000"])
        plt.scatter(X[:, 0], X[:, 1], s=10, color=colors[y_pred])

        plt.xlim(-2.5, 2.5)
        plt.ylim(-2.5, 2.5)
        plt.xticks(())
        plt.yticks(())
        plt.text(.99, .01, ('%.2fs' % (t1 - t0)).lstrip('0'),
                 transform=plt.gca().transAxes, size=15,
                 horizontalalignment='right')
        plot_num += 1

plt.show()